In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Apr_dict_JSON.json', 'r') as json_file:
    link_day_minute_Apr_dict_JSON = json.load(json_file)

In [18]:
#weekend_Apr_list = [1, 7, 8, 14, 15, 21, 22, 28, 29]
weekend_Apr_list = [1, 8, 15, 22, 29]

In [19]:
link_day_minute_Apr_list = []
for link_idx in range(24):
    for day in weekend_Apr_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Apr_list.append(link_day_minute_Apr_dict_JSON[key] ['AM_flow_minute'][minute_idx])

In [20]:
len(link_day_minute_Apr_list)

14400

In [21]:
x = np.matrix(link_day_minute_Apr_list)
x = np.matrix.reshape(x, 24, 600)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [22]:
x[:,:2]

matrix([[ 1287.28948464,  1271.34101398],
        [ 1213.522316  ,  1254.38538945],
        [ 1556.31480605,  1542.76934181],
        [ 1570.56378118,  1592.29391951],
        [ 1534.11012025,  1561.65513209],
        [ 1579.87929533,  1598.10809301],
        [ 1308.3285316 ,  1310.37496981],
        [ 1257.64327598,  1283.19889696],
        [ 1664.02988359,  1668.86194482],
        [ 1669.96452786,  1665.96202488],
        [ 1445.05731617,  1427.55655979],
        [ 1257.52210986,  1284.88655777],
        [ 1129.07710533,  1147.25662807],
        [ 1367.91027111,  1266.27863782],
        [ 1208.9939992 ,  1222.26082272],
        [ 1013.50767283,   919.17982698],
        [ 1274.76058234,  1261.49406794],
        [ 1313.79783333,  1288.06853512],
        [ 1568.33747432,  1495.2566459 ],
        [ 1564.0549542 ,  1498.07658732],
        [ 1394.98827601,  1419.30274259],
        [ 1320.61203112,  1407.91038881],
        [ 1298.99132597,  1302.07907642],
        [ 1307.31011551,  1289.267

In [23]:
np.size(x, 0), np.size(x, 1)

(24, 600)

In [24]:
np.size(A,0), np.size(A,1)

(24, 140)

In [25]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [9e+00, 3e+03]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 27
 AA' NZ     : 3.510e+02
 Factor NZ  : 3.780e+02
 Factor Ops : 6.930e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -3.58743869e+07  0.00000000e+00  5.98e+03 1.09e+03  1.00e+06     0s
   1  -7.81216974e+06 -8.89805032e+06  5.62e+02 1.03e+02  1.89e+05     0s
   2  -5.43615933e+06 -6.62008136e+06  1.03e+01 1.88e+00  2.35e+04     0s
   3  -5.57013178e+06 -5.66012165e+06  1.81e-03 3.31e-04  1.6

In [26]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Apr_weekend.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [27]:
lam_list

[2.305513995101105e-12,
 1277.3437539715562,
 2.5326551013852442e-12,
 608.4965888047899,
 98.46299696262315,
 330.52031433570824,
 7.642767604634633e-10,
 759.8072933402378,
 1.6631460286263867e-10,
 1218.1944218039632,
 1.6631460287889814e-10,
 1.6631460286964985e-10,
 1.663146028626978e-10,
 1.6726832626736298e-10,
 1915.8954659493352,
 783.6444580976985,
 142.564184684521,
 135.0218465811858,
 497.44432005524607,
 260.4285275724006,
 283.7218433700526,
 2.037762686860875e-05,
 1188.9560921147047,
 1270.0123152707204,
 0.051760591790202426,
 0.11123272924518801,
 0.045260833927113,
 1072.7566596246083,
 0.03142300521284591,
 103.34984717231433,
 0.04577202738103403,
 0.06113449188906839,
 1044.616099687717,
 563.2966023460996,
 173.2536289370425,
 0.1059112608817383,
 12.009389413380276,
 960.2106754249772,
 938.1822277565511,
 0.11691509213606777,
 866.0574979638762,
 192.60789343670626,
 2.3507254350347455e-06,
 2.3507511708667427e-06,
 2.3507492248021767e-06,
 205.020308572559,
 